# Lab 2020-11-13: Satisfiability Modulo Theories

During this lab you will be using Z3 prover to deduce facts and solve combinatorial problems.


## Using Z3 in Jupyter

Code cells are executed by the selected kernel (Python 3, based on [IPython](https://ipython.readthedocs.io/en/stable/)), which provides [Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to a better interaction with the environment. In particular we'll use `%%script` command which works like the [Unix Shebang](https://en.wikipedia.org/wiki/Shebang_(Unix)) passing the content of the cell to the executing program as standard input.


In [1]:
%%script z3 -in -smt2

(declare-const p Bool)
(declare-const q Bool)
(declare-const r Bool)
(define-fun conjecture () Bool
    (=> (and (=> p q) (=> q r))
        (=> p r)))
(assert (not conjecture))
(check-sat)

unsat


To ease the interaction with Z3 we can define a macro for the `%%script z3 -in -smt2` header using the [%alias](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-alias) line magic:

In [2]:
%alias_magic -c z3 script -p "z3 -in -smt2"

Created `%%z3` as an alias for `%%script z3 -in -smt2`.


Now you can use `%%z3` instead of the `%%script`:

In [3]:
%%script z3 -in -smt2

(declare-const p Bool)
(declare-const q Bool)
(declare-const r Bool)
(define-fun conjecture () Bool
    (=> (and (=> p q) (=> q r))
        (=> p r)))
(assert (not conjecture))
(check-sat)

unsat


## Z3 as SAT solver

Z3 can be used to solve Boolean Satisfiability (SAT) problems. The [Z3 tutorial](https://rise4fun.com/z3/tutorialcontent/guide#h22) provides a quick introduction to SMT-LIB syntax.



### Unicorn example

Consider the unicorn example introduced during the lecture:

-   If the unicorn is mythical, then it is immortal
-   If the unicorn is not mythical, then it is a mortal mammal
-   If the unicorn is either immortal or a mammal, then it is horned
-   The unicorn is magical if it is horned

Abstracting the domain using propositional variables

-   $m$: the unicorn is mythical
-   $i$: the unicorn is immortal
-   $l$: the unicorn is mammal
-   $h$: the unicorn is horned
-   $g$: the unicorn is magical

the above statements became

$$(m\rightarrow i)\wedge(\neg m \rightarrow (\neg i \wedge l))\wedge((i \vee l) \rightarrow h)\wedge(h\rightarrow g)$$

Use the above formula and Z3 to answer the following three questions from the lecture. You should write the SMT-LIB code and explain it.

1.  Is the unicorn mythical?

In [4]:
%%script z3 -in -smt2

(declare-const m Bool)
(declare-const i Bool)
(declare-const l Bool)
(declare-const h Bool)
(declare-const g Bool)
(assert (and (=> m i) (=> (not m) (and (not i) l)) (=> (or i l) h) (=> h g)))
(assert (= m true))
(check-sat)

sat


In [5]:
%%script z3 -in -smt2

(declare-const m Bool)
(declare-const i Bool)
(declare-const l Bool)
(declare-const h Bool)
(declare-const g Bool)
(assert (and (=> m i) (=> (not m) (and (not i) l)) (=> (or i l) h) (=> h g)))
(assert (= m false))
(check-sat)

sat


2.  Is it magical?

In [6]:
%%script z3 -in -smt2

(declare-const m Bool)
(declare-const i Bool)
(declare-const l Bool)
(declare-const h Bool)
(declare-const g Bool)
(assert (and (=> m i) (=> (not m) (and (not i) l)) (=> (or i l) h) (=> h g)))
(assert (= g true))
(check-sat)

sat


In [7]:
%%script z3 -in -smt2

(declare-const m Bool)
(declare-const i Bool)
(declare-const l Bool)
(declare-const h Bool)
(declare-const g Bool)
(assert (and (=> m i) (=> (not m) (and (not i) l)) (=> (or i l) h) (=> h g)))
(assert (= g false))
(check-sat)

unsat


3.  Is it horned?

In [8]:
%%script z3 -in -smt2

(declare-const m Bool)
(declare-const i Bool)
(declare-const l Bool)
(declare-const h Bool)
(declare-const g Bool)
(assert (and (=> m i) (=> (not m) (and (not i) l)) (=> (or i l) h) (=> h g)))
(assert (= h true))
(check-sat)


sat


In [9]:
%%script z3 -in -smt2

(declare-const m Bool)
(declare-const i Bool)
(declare-const l Bool)
(declare-const h Bool)
(declare-const g Bool)
(assert (and (=> m i) (=> (not m) (and (not i) l)) (=> (or i l) h) (=> h g)))
(assert (= h false))
(check-sat)


unsat


### Unicorn conclusions

By using the Z3 solver we can deduce that the unicorn can be mythical or not mythical and for sure magical and horned.

### Graph colouring

In the previous exercise we looked at SAT as a way to answer to questions; i.e. by focusing on logical deduction. However SAT can be used to find solutions to combinatorial (NP) problems. In this exercise we will consider the [Graph Colouring problem](https://en.wikipedia.org/wiki/Graph_coloring).

As seen in the lecture Graph Colouring can be used to solve different problems (e.g. [scheduling](https://en.wikipedia.org/wiki/Graph_coloring#Scheduling)).

Consider the following problem by James L. Hein:

> Some people form committees to do various tasks. The problem is to schedule the committee meetings in as few time slots as possible.
> We'll represent each person with a number. For example, let $S = \{1, 2, 3, 4, 5, 6, 7\}$ represent a set of seven people, and suppose they have formed six three-person committees as follows:
>
> $$S_1 = \{1, 2, 3\}, S_2 = \{2, 3, 4\}, S_3 = \{3, 4, 5\}, S_4 = \{4, 5, 6\}, S_5 = \{5, 6, 7\}, S_6 = \{1, 6, 7\}$$
>
> We can model the problem with a graph, where the committee names are the vertices and an edge connects two vertices if a person belongs to both committees represented by the vertices.
> If each committee meets for one hour, what is the smallest number of hours needed for the committees to do their work?
> From the graph, it follows that an edge between two committees means that they have at least one member in common. Thus, they cannot meet at the same time. No edge between committees means that they can meet at the same time. For example, committees $S_1$ and $S_4$ can meet at the same hour.

Solve the problem using a graph colouring problem with Z3.


In [10]:
%%script z3 -in -smt2

;General instructions on ho to construct clauses and constant can be taken from 
;chapter n. 4 of this document http://ceur-ws.org/Vol-533/09_LANMR09_06.pdf
;another explanation on how to construct clauses is: https://airccj.org/CSCP/vol3/csit3213.pdf
        
;check if can be satisfied in 1 hour (k=1)

(set-logic QF_LIA)
(set-option :produce-models true)

;define all the comitees S[i, k] with the hour k 1-3
(declare-const S11 Bool)
(declare-const S21 Bool)
(declare-const S31 Bool)
(declare-const S41 Bool)
(declare-const S51 Bool)
(declare-const S61 Bool)

;assert that every comitee sik has at least 1 hour k
(assert S11)
(assert S21)
(assert S31)
(assert S41)
(assert S51)
(assert S61)

;check that if there is a comitee S[i, k] there is no connected comittee S[j, k] with same hour
(assert (=> S11 (not (or S21 S31 S61) )))
(assert (=> S21 (not (or S11 S31 S41) )))
(assert (=> S31 (not (or S11 S21 S41 S51) )))
(assert (=> S41 (not (or S21 S31 S51 S61) )))
(assert (=> S51 (not (or S31 S41 S61) )))
(assert (=> S61 (not (or S11 S41 S51) )))

(check-sat)
(exit)

unsat


In [11]:
%%script z3 -in -smt2

;check if can be satisfied in 2 hour (k=2)

(set-logic QF_LIA)
(set-option :produce-models true)

;define all the comitees S[i, k] with the hour k 1-3
(declare-const S11 Bool)
(declare-const S12 Bool)
(declare-const S21 Bool)
(declare-const S22 Bool)
(declare-const S31 Bool)
(declare-const S32 Bool)
(declare-const S41 Bool)
(declare-const S42 Bool)
(declare-const S51 Bool)
(declare-const S52 Bool)
(declare-const S61 Bool)
(declare-const S62 Bool)

;assert that every comitee S[i, k] has been assigned at least 1 hour k
(assert (or S11 S12 ))
(assert (or S21 S22 ))
(assert (or S31 S32 ))
(assert (or S41 S42 ))
(assert (or S51 S52 ))
(assert (or S61 S62 ))

;assert that every committee S[i, k] has been assigned at most 1 hour k
(assert (=> S11 (not (or S12) )))
(assert (=> S12 (not (or S11) )))
(assert (=> S21 (not (or S22) )))
(assert (=> S22 (not (or S21) )))
(assert (=> S31 (not (or S32) )))
(assert (=> S32 (not (or S31) )))
(assert (=> S41 (not (or S42) )))
(assert (=> S42 (not (or S41) )))
(assert (=> S51 (not (or S52) )))
(assert (=> S52 (not (or S51) )))
(assert (=> S61 (not (or S62) )))
(assert (=> S62 (not (or S61) )))

;check that if there is a comitee S[i, k] there is no connected comittee S[j, k] with same hour k
(assert (=> S11 (not (or S21 S31 S61) )))
(assert (=> S12 (not (or S22 S32 S62) )))

(assert (=> S21 (not (or S11 S31 S41) )))
(assert (=> S22 (not (or S12 S32 S42) )))

(assert (=> S31 (not (or S11 S21 S41 S51) )))
(assert (=> S32 (not (or S12 S22 S42 S52) )))

(assert (=> S41 (not (or S21 S31 S51 S61) )))
(assert (=> S42 (not (or S22 S32 S52 S62) )))

(assert (=> S51 (not (or S31 S41 S61) )))
(assert (=> S52 (not (or S32 S42 S62) )))

(assert (=> S61 (not (or S11 S41 S51) )))
(assert (=> S62 (not (or S12 S42 S52) )))

(check-sat)

(exit)

unsat


In [12]:
%%script z3 -in -smt2

;check if can be satisfied in 3 hour (k=3)

(set-logic QF_LIA)
(set-option :produce-models true)

;define all the comitees S[i, k] with the hour k 1-3
(declare-const S11 Bool)
(declare-const S12 Bool)
(declare-const S13 Bool)
(declare-const S21 Bool)
(declare-const S22 Bool)
(declare-const S23 Bool)
(declare-const S31 Bool)
(declare-const S32 Bool)
(declare-const S33 Bool)
(declare-const S41 Bool)
(declare-const S42 Bool)
(declare-const S43 Bool)
(declare-const S51 Bool)
(declare-const S52 Bool)
(declare-const S53 Bool)
(declare-const S61 Bool)
(declare-const S62 Bool)
(declare-const S63 Bool)

;assert that every comitee S[i, k] has been assigned at least 1 hour k
(assert (or S11 S12 S13))
(assert (or S21 S22 S23))
(assert (or S31 S32 S33))
(assert (or S41 S42 S43))
(assert (or S51 S52 S53))
(assert (or S61 S62 S63))

;assert that every committee S[i, k] has been assigned at most 1 hour k
(assert (=> S11 (not (or S12 S13) )))
(assert (=> S12 (not (or S11 S13) )))
(assert (=> S13 (not (or S11 S12) )))
(assert (=> S21 (not (or S22 S23) )))
(assert (=> S22 (not (or S21 S23) )))
(assert (=> S23 (not (or S21 S22) )))
(assert (=> S31 (not (or S32 S33) )))
(assert (=> S32 (not (or S31 S33) )))
(assert (=> S33 (not (or S31 S32) )))
(assert (=> S41 (not (or S42 S43) )))
(assert (=> S42 (not (or S41 S43) )))
(assert (=> S43 (not (or S41 S42) )))
(assert (=> S51 (not (or S52 S53) )))
(assert (=> S52 (not (or S51 S53) )))
(assert (=> S53 (not (or S51 S52) )))
(assert (=> S61 (not (or S62 S63) )))
(assert (=> S62 (not (or S61 S63) )))
(assert (=> S63 (not (or S61 S62) )))

;check that if there is a comitee S[i, k] there is no connected comittee S[j, k] with same hour k
(assert (=> S11 (not (or S21 S31 S61) )))
(assert (=> S12 (not (or S22 S32 S62) )))
(assert (=> S13 (not (or S23 S33 S63) )))

(assert (=> S21 (not (or S11 S31 S41) )))
(assert (=> S22 (not (or S12 S32 S42) )))
(assert (=> S23 (not (or S13 S33 S43) )))

(assert (=> S31 (not (or S11 S21 S41 S51) )))
(assert (=> S32 (not (or S12 S22 S42 S52) )))
(assert (=> S33 (not (or S13 S23 S43 S53) )))

(assert (=> S41 (not (or S21 S31 S51 S61) )))
(assert (=> S42 (not (or S22 S32 S52 S62) )))
(assert (=> S43 (not (or S23 S33 S53 S63) )))

(assert (=> S51 (not (or S31 S41 S61) )))
(assert (=> S52 (not (or S32 S42 S62) )))
(assert (=> S53 (not (or S33 S43 S63) )))

(assert (=> S61 (not (or S11 S41 S51) )))
(assert (=> S62 (not (or S12 S42 S52) )))
(assert (=> S63 (not (or S13 S43 S53) )))

(check-sat)
(get-value (S11 S12 S13 S21 S22 S23 S31 S32 S33 S41 S42 S43 S51 S52 S53 S61 S62 S63))
(exit)

sat
((S11 true)
 (S12 false)
 (S13 false)
 (S21 false)
 (S22 true)
 (S23 false)
 (S31 false)
 (S32 false)
 (S33 true)
 (S41 true)
 (S42 false)
 (S43 false)
 (S51 false)
 (S52 true)
 (S53 false)
 (S61 false)
 (S62 false)
 (S63 true))


In [14]:
%%script z3 -in -smt2

; DISCLAIMER THIS KIND OF MODELING IS NOT RECOMMENDED TO RESOLVE SAT PROBLEMS

(set-option :smt.auto-config false) ; disable automatic self configuration
(set-option :smt.mbqi false) ; disable model-based quantifier instantiation

(set-option :produce-models true)

(define-fun maxp () Int 7)
(define-fun maxc () Int 6)
(define-fun maxg () Int 2)

(declare-fun personInCommittee (Int Int) Bool)
(declare-fun committeeInGroup (Int Int) Bool)

(define-fun legalPerson ((x Int)) Bool
     (and
         (< x maxp)
         (>= x 0)
     )
)

(define-fun legalCommittee ((x Int)) Bool
     (and
         (< x maxc)
         (>= x 0)
     )
)

(define-fun legalGroup ((x Int)) Bool
     (and
         (< x maxg)
         (>= x 0)
     )
)

(define-fun distinctPeople ((c1 Int) (c2 Int)) Bool
    (forall ((a Int)) (and (not 
                                (and (personInCommittee a c1) 
                                     (personInCommittee a c2)
                                )
                           )
                           (legalPerson a)
                      )
    )
)

(assert (<= maxg 10))

(assert (and (personInCommittee 0 0) (personInCommittee 0 5)
             (personInCommittee 1 0) (personInCommittee 1 1)
             (personInCommittee 2 0) (personInCommittee 2 1) (personInCommittee 2 2)
             (personInCommittee 3 1) (personInCommittee 3 2) (personInCommittee 3 3)
             (personInCommittee 4 2) (personInCommittee 4 3) (personInCommittee 4 4)
             (personInCommittee 5 3) (personInCommittee 5 4) (personInCommittee 5 5)
             (personInCommittee 6 4) (personInCommittee 6 5)
        )
)

(assert (not 
           (or
                (personInCommittee 0 1) (personInCommittee 0 2) (personInCommittee 0 3) (personInCommittee 0 4) 
                (personInCommittee 1 2) (personInCommittee 1 3) (personInCommittee 1 4) (personInCommittee 1 5)
                (personInCommittee 2 3) (personInCommittee 2 4) (personInCommittee 2 5)
                (personInCommittee 3 0) (personInCommittee 3 4) (personInCommittee 3 5)
                (personInCommittee 4 0) (personInCommittee 4 1) (personInCommittee 4 5)
                (personInCommittee 5 0) (personInCommittee 5 1) (personInCommittee 5 2)
                (personInCommittee 6 0) (personInCommittee 6 1) (personInCommittee 6 2) (personInCommittee 6 3)
           )
        )
)

(assert
    (forall ((g Int) (c1 Int)) (and
                                   (legalGroup g)
                                   (legalCommittee c1)
                                   (=> (committeeInGroup c1 g) (forall ((c2 Int)) (and (legalCommittee c2) (not (= c2 c1)) (or (distinctPeople c1 c2) (not (committeeInGroup c2 g))))))
                                   (=> (forall ((c2 Int)) (and (legalCommittee c2) (not (= c2 c1)) (or (distinctPeople c1 c2) (not (committeeInGroup c2 g))))) (committeeInGroup c1 g))
                                )
    )
)
                             
(check-sat)
;(get-model)

(exit)

unknown


## Z3 as SMT solver

Encode the following formulae and verify their satisfiability. The formulae below are written using an *abstract notation* and should be adapted to SMT-LIB language. For each formula you should identify the right SMT-LIB *Logic* to be used. Z3 should be smart enough to detect the fragment without the SMT-LIB declaration, but it's not guaranteed that the guess is correct.

### [Arithmetic](https://rise4fun.com/z3/tutorialcontent/guide#h24)

$$(x - y \leq 0) \land (y - z \leq 0) \land ((z - x \leq -1) \lor (z - x \leq -2))$$

$$(b \lor (x + y \leq 0)) \land (\neg b \lor (x + z \leq 10))$$

In [16]:
%%script z3 -in -smt2

(set-logic QF_LIA)
(set-option :produce-models true)

(declare-const x Int)
(declare-const y Int)
(declare-const z Int)

(assert (<= (- x y) 0))
(assert (<= (- y z) 0))
(assert (or (<= (- z x) -1) (<= (- z x) -2)))

(check-sat)
(exit)

unsat


In [17]:
%%script z3 -in -smt2

;(set-logic QF_LIA)
(set-option :produce-models true)

(declare-const b Bool)
(declare-const x Int)
(declare-const y Int)
(declare-const z Int)

(assert (and (or b (<= (+ x y) 0)) (or (not b) (<= (+ x z) 10))))

(check-sat)
(get-value (b x y z))
(exit)

sat
((b false)
 (x 0)
 (y 0)
 (z 11))


### [Arrays](https://rise4fun.com/z3/tutorialcontent/guide#h26)

$$write(a,i,x)\neq b \land read(b,i) = y \land read(write(b,i,x), j) = y \land a=b \land  i=j$$

Note that in SMT-LIB *read* and *write* are called *select* and *store*.

In [18]:
%%script z3 -in -smt2

(declare-const i Int)
(declare-const j Int)
(declare-const x Int)
(declare-const y Int)
(declare-const a (Array Int Int))
(declare-const b (Array Int Int))

(assert
    (and
        (not (= (store a i x) b))
        (= (select b i) y)
        (= (select (store b i x) j) y)
        (= a b)
        (= i j)
    )
)

(check-sat)
(exit)

unsat


### [Bit vectors](https://rise4fun.com/z3/tutorialcontent/guide#h25)

In the formula below the vectors size must be bigger or equal than 2. The `a[.:.]` denotes slicing (in SMTLIB it's the `extract` function), `|` bitwise OR operation, and `+` addition where arrays are interpreted as (unsigned) integers binary encoded. The syntax for bit vectors assertions can be confusing, see [this document](https://stp.readthedocs.io/en/latest/smt-input-language.html) for more details.

$$a[0:1]\neq b[0:1] \land (a|b)=c \land c[0]=0 \land a[1]+b[1]=0$$

In [19]:
%%script z3 -in -smt2

(set-logic QF_BV)
(set-info :smt-lib-version 2.0)

(declare-const a (_ BitVec 3))
(declare-const b (_ BitVec 3))
(declare-const c (_ BitVec 3))

(assert (and
            (not (= ((_ extract 1 0) a) ((_ extract 1 0) b)))
            (= (bvor a b) c)
            (= ((_ extract 0 0) c) #b0)
            (= (bvadd ((_ extract 1 1) a) ((_ extract 1 1) b)) #b0)
        )
)

(check-sat)
(exit)

unsat


### Combine theories

$$a=b+2 \land A=write(B,a+1,4) \land (read(A,b+3)=2 \lor f(a-1)\neq f(b+1))$$

In [20]:
%%script z3 -in -smt2

(declare-const a Int)
(declare-const b Int)
(declare-const A (Array Int Int))
(declare-const B (Array Int Int))
(declare-fun f (Int) Int)

(assert
    (and
        (= a (+ b 2))
        (= A (store B (+ a 1) 4))
        (or (= (select A (+ b 3)) 2) (not (= (f (- a 1)) (f (+ b 1)))))
    )
)
     
(check-sat)
(exit)

unsat


### EUF and weaker theories

Use a SMT solver to verify that the formula

$$(v_1 \leq v_2)\land(v_2 \leq v_3)\rightarrow (v_1 \leq v_3)$$

is a tautology in a logic with inequality (e.g. [QF\_LIA](http://smtlib.cs.uiowa.edu/logics-all.shtml#QF_LIA) or [QF\_IDL](http://smtlib.cs.uiowa.edu/logics-all.shtml#QF_IDL)) but not in a theory of uninterpreted functions; e.g. [QF\_UF](http://smtlib.cs.uiowa.edu/logics-all.shtml#QF_UF) (mind that in SMT-LIB you should declare functions if they are not defined in the logic).

Notes:

1.  You should use `set-logic` to specify the logic.
2.  In [QF\_UF](http://smtlib.cs.uiowa.edu/logics-all.shtml#QF_UF) you don't have less or equal (`<=`) nor a domain (sort) to use, so:
    -   define a new sort with `(declare-sort name 0)` where `name` is the name you select
    -   define less or equal (`<=`) with `(declare-fun <= (name name) Bool)`

In [21]:
%%script z3 -in -smt2

(set-logic QF_UF)

(declare-sort MyType 0)

(declare-fun <= (MyType MyType) Bool)

(declare-const v1 MyType)
(declare-const v2 MyType)
(declare-const v3 MyType)

; WE LOOKED FOR A COUNTEREXAMPLE TO NEGATE THAT THE FORMULA IS A TAUTOLOGY 

(assert
    (not (=> (and 
                 (<= v1 v2)
                 (<= v2 v3)
             )
             (<= v1 v3)
         )
    )
)
 
(check-sat)
(get-model)
(exit)

sat
(model 
  ;; universe for MyType:
  ;;   MyType!val!2 MyType!val!0 MyType!val!1 
  ;; -----------
  ;; definitions for universe elements:
  (declare-fun MyType!val!2 () MyType)
  (declare-fun MyType!val!0 () MyType)
  (declare-fun MyType!val!1 () MyType)
  ;; cardinality constraint:
  (forall ((x MyType))
          (or (= x MyType!val!2) (= x MyType!val!0) (= x MyType!val!1)))
  ;; -----------
  (define-fun v2 () MyType
    MyType!val!1)
  (define-fun v3 () MyType
    MyType!val!2)
  (define-fun v1 () MyType
    MyType!val!0)
  (define-fun <= ((x!0 MyType) (x!1 MyType)) Bool
    (ite (and (= x!0 MyType!val!0) (= x!1 MyType!val!2)) false
      true))
)


## Minesweeper

Let's consider the [Minesweeper](https://en.wikipedia.org/wiki/Minesweeper_(video_game)) game, where you have a grid of covered squares, some of which contain mines, but you don't know which. Your job is to uncover every square which does not contain a mine. If you uncover a square containing a mine, you lose. If you uncover a square which does not contain a mine, you are told how many mines are contained within the eight surrounding squares.

Once you start unveiling the minefield you can deduce the location of mines by solving a system of equations by trying to *disprove* that in a specific cell you can place a mine. In order to do that you can represent each cell by an integer variable that contains the number of mines in the specific cell (0 or 1).

For this exercise we will use a problem generator with the property that the generated problems can be always solved by deduction (mind that in general this is not the case). The generator is available on the [Simon Tatham's Portable Puzzle Collection](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html) page:

> The first square you open is guaranteed to be safe, and (by default) you are guaranteed to be able to solve the whole grid by deduction rather than guesswork. (Deductions may require you to think about the total number of mines.)

An interesting instance you can try to solve is the following:

[![sweeper example](media/sweeper_6x6-10.png)](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html#6x6:2,2,m1bf215cfe)

and you can use [this link](https://www.chiark.greenend.org.uk/~sgtatham/puzzles/js/mines.html#6x6:2,2,m1bf215cfe) to verify your solution. You should include the SMT-LIB code you're using to solve the instance.

A convenient representation for the problem (useful later on) is a matrix of numbers representing the number of mines *seen* by the cell or '?' for the unknown cells. E.g the above example would correspond to

    ??????
    ?1123?
    ?1003?
    ?1002?
    ?2234?
    ??????

In [22]:
%run minesweeper_solver "world0/world0_0.json" "world0/world0_0_result.json"

Input matrix:

??????
?1123?
?1003?
?1002?
?2234?
??????

Resulting matrix:

ssXsXs
s1123X
s1003X
X1002X
s2234s
ssXXXX


#### Solving Minesweeper manually creating all clauses

    001?
    012?
    01??
    01??

In [23]:
%%script z3 -in -smt2

; minewsweeper configuration is different than assignment, in order to keep problem small
; 0 0 1 ?
; 0 1 2 ?
; 0 1 ? ?
; 0 1 ? ?

(set-option :produce-models true)

; declare all cells as constants
(declare-const m00 Int)
(declare-const m01 Int)
(declare-const m02 Int)
(declare-const m03 Int)
(declare-const m10 Int)
(declare-const m11 Int)
(declare-const m12 Int)
(declare-const m13 Int)
(declare-const m20 Int)
(declare-const m21 Int)
(declare-const m22 Int)
(declare-const m23 Int)
(declare-const m30 Int)
(declare-const m31 Int)
(declare-const m32 Int)
(declare-const m33 Int)

; each of the cell of the minesweeper can be a mine (1) or not (0)
(assert (and (<= m00 1) (>= m00 0)
             (<= m01 1) (>= m01 0)
             (<= m02 1) (>= m02 0)
             (<= m03 1) (>= m03 0)
             (<= m10 1) (>= m10 0)
             (<= m11 1) (>= m11 0)
             (<= m12 1) (>= m12 0)
             (<= m13 1) (>= m13 0)
             (<= m20 1) (>= m20 0)
             (<= m21 1) (>= m21 0)
             (<= m22 1) (>= m22 0)
             (<= m23 1) (>= m23 0)
             (<= m30 1) (>= m30 0)
             (<= m31 1) (>= m31 0)
             (<= m32 1) (>= m32 0)
             (<= m33 1) (>= m33 0)
        )
)

; total mines in the world are 3
(assert (= 3 (+ m00 (+ m01 (+ m02 (+ m03 (+ m10 (+ m11 (+ m12 (+ m13 
             (+ m20 (+ m21 (+ m22 (+ m23 (+ m30 (+ m31 (+ m32 m33) ))))))))))))))))

;sum the adjacent (also diagonal) cells in the world and assign them the value of the neibouring sum 
(assert (= 0 (+ m10 (+ m01 m11))))
(assert (= 0 (+ m00 (+ m20 (+ m11 (+ m01 m21))))))
(assert (= 1 (+ m10 (+ m30 (+ m21 (+ m11 m31))))))
(assert (= 0 (+ m00 (+ m02 (+ m11 m10 m12 )))))
(assert (= 1 (+ m01 (+ m21 (+ m10 (+ m12 (+ m00 (+ m20 (+ m02 m22)))))))))
(assert (= 2 (+ m20 (+ m22 (+ m11 (+ m31 (+ m10 (+ m12 (+ m30 m32)))))))))
(assert (= 0 (+ m01 (+ m03 (+ m12 (+ m11 m13))))))
(assert (= 1 (+ m11 (+ m13 (+ m02 (+ m22 (+ m01 (+ m03 (+ m21 m23)))))))))
(assert (= 0 (+ m02 (+ m13 m12))))
(assert (= 1 (+ m03 (+ m23 (+ m12 (+ m02 m22))))))

(check-sat)
(get-value (m00 m01 m02 m03 m10 m11 m12 m13 m20 m21 m22 m23 m30 m31 m32 m33))
(exit)

sat
((m00 0)
 (m01 0)
 (m02 0)
 (m03 0)
 (m10 0)
 (m11 0)
 (m12 0)
 (m13 0)
 (m20 0)
 (m21 0)
 (m22 1)
 (m23 0)
 (m30 1)
 (m31 0)
 (m32 0)
 (m33 1))


#### Step by step resolving a minesweeper problem in world1/world1_0.json

Number of mines: 12

Size: 10 x 6

Configuration:

    ??20001???
    ??20012???
    ?31001????
    ?100012???
    ?111001221
    ???1000000

In [24]:
%run minesweeper_solver "world1/world1_0.json" "world1/world1_0_result.json"

Input matrix:

??20001???
??20012???
?31001????
?100012???
?111001221
???1000000

Resulting matrix:

?X20001???
XX20012???
X31001Xs??
s100012XXs
s111001221
ssX1000000


In [25]:
%run minesweeper_solver "world1/world1_1.json" "world1/world1_1_result.json"

Input matrix:

?X20001???
XX20012???
X31001X4??
1100012XX1
0111001221
01X1000000

Resulting matrix:

?X20001???
XX20012???
X31001X4ss
1100012XX1
0111001221
01X1000000


In [26]:
%run minesweeper_solver "world1/world1_2.json" "world1/world1_2_result.json"

Input matrix:

?X20001???
XX20012???
X31001X443
1100012XX1
0111001221
01X1000000

Resulting matrix:

?X20001X??
XX20012sXX
X31001X443
1100012XX1
0111001221
01X1000000


In [27]:
%run minesweeper_solver "world1/world1_3.json" "world1/world1_3_result.json"

Input matrix:

?X20001X??
XX200123XX
X31001X443
1100012XX1
0111001221
01X1000000

Resulting matrix:

?X20001Xs?
XX200123XX
X31001X443
1100012XX1
0111001221
01X1000000


In [28]:
%run minesweeper_solver "world1/world1_4.json" "world1/world1_4_result.json"

Input matrix:

?X20001X3?
XX200123XX
X31001X443
1100012XX1
0111001221
01X1000000

Resulting matrix:

XX20001X3s
XX200123XX
X31001X443
1100012XX1
0111001221
01X1000000


In [29]:
%run minesweeper_solver "world1/world1_5.json" "world1/world1_5_result.json"

Input matrix:

XX20001X32
XX200123XX
X31001X443
1100012XX1
0111001221
01X1000000

Resulting matrix:

XX20001X32
XX200123XX
X31001X443
1100012XX1
0111001221
01X1000000


#### Example of a world that is not resolvable in deterministic way (world2/world2_0.json)

Number of mines: 10

Size: 9 x 6

Configuration:

    ??2??11??
    ??4321122
    2??100000
    122100111
    1100001?1
    ?10000111

In [30]:
%run minesweeper_solver "world2/world2_0.json" "world2/world2_0_result.json"

Input matrix:

??2??11??
??4321122
2??100000
122100111
1100001?1
?10000111

Resulting matrix:

??2XX11XX
??4321122
2XX100000
122100111
1100001X1
X10000111


## Deduction in the Wumpus world

In this exercise you should consider the [Hunt the Wumpus](https://docs.google.com/document/d/1ySK0M-txOuIVWUGxPB02Ws_p5nSi5aLMlDQl4NMtAlk/edit?usp=sharing) domain, and use the prover to deduce facts about the given instance of the problem.

Use the techniques that you learned in the previous exercises to deduce (if possible) the safe cells and the position of pits and Wumpus given the perceptions that the agent would detect from a given cell.

The input is given as a matrix where cells contain either the letter `B` (breeze) or `S` (stench) according whether the agent, placed in the corresponding cell, would feel one of the two situations. We assume that the sensing information *doesn't* include the presence of pit and wumpus in the cells where they are located (see the example below for details).

E.g. a possible configuration and the corresponding "perception" matrix would be:

    +---+---+---+---+        +---+---+---+---+
    |  P| G |   |   |        |   |  B|S  |   |
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+        +---+---+---+---+
    |   |   |W  |   |        |  B|S  |   |S B|
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+ =====> +---+---+---+---+
    |   |   |   |  P|        |   |   |S B|   |
    |   |   |   |   |        |   |   |   |   |
    +---+---+---+---+        +---+---+---+---+
    |   |   |   |   |        |   |   |   |  B|
    | A |   |   |   |        | A |   |   |   |
    +---+---+---+---+        +---+---+---+---+

Note that neither the cells where pits and wumpus are located don't sense breeze or stench coming from the same cell. They would sense neighbouring cells. 


Try to solve the following two examples by encoding using SMT-LIB and explain the method you used.

1. **Example 1**
    ```
    +---+---+---+---+
    |   |  B|  B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |   |  B|  B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|   |  B|S B|
    |   |   |   |   |
    +---+---+---+---+
    |   |  B|S  |   |
    | A |   |   |   |
    +---+---+---+---+
    ```

In [31]:
%%script z3 -in -smt2

(set-option :produce-models true)

(declare-const p00 Int)
(declare-const p01 Int)
(declare-const p02 Int)
(declare-const p03 Int)
(declare-const p10 Int)
(declare-const p11 Int)
(declare-const p12 Int)
(declare-const p13 Int)
(declare-const p20 Int)
(declare-const p21 Int)
(declare-const p22 Int)
(declare-const p23 Int)
(declare-const p30 Int)
(declare-const p31 Int)
(declare-const p32 Int)
(declare-const p33 Int)

(declare-const w00 Int)
(declare-const w01 Int)
(declare-const w02 Int)
(declare-const w03 Int)
(declare-const w10 Int)
(declare-const w11 Int)
(declare-const w12 Int)
(declare-const w13 Int)
(declare-const w20 Int)
(declare-const w21 Int)
(declare-const w22 Int)
(declare-const w23 Int)
(declare-const w30 Int)
(declare-const w31 Int)
(declare-const w32 Int)
(declare-const w33 Int)

(assert (and (<= p00 1) (>= p00 0)
             (<= p01 1) (>= p01 0)
             (<= p02 1) (>= p02 0)
             (<= p03 1) (>= p03 0)
             (<= p10 1) (>= p10 0)
             (<= p11 1) (>= p11 0)
             (<= p12 1) (>= p12 0)
             (<= p13 1) (>= p13 0)
             (<= p20 1) (>= p20 0)
             (<= p21 1) (>= p21 0)
             (<= p22 1) (>= p22 0)
             (<= p23 1) (>= p23 0)
             (<= p30 1) (>= p30 0)
             (<= p31 1) (>= p31 0)
             (<= p32 1) (>= p32 0)
             (<= p33 1) (>= p33 0)
        )
)

(assert (and (<= w00 1) (>= w00 0)
             (<= w01 1) (>= w01 0)
             (<= w02 1) (>= w02 0)
             (<= w03 1) (>= w03 0)
             (<= w10 1) (>= w10 0)
             (<= w11 1) (>= w11 0)
             (<= w12 1) (>= w12 0)
             (<= w13 1) (>= w13 0)
             (<= w20 1) (>= w20 0)
             (<= w21 1) (>= w21 0)
             (<= w22 1) (>= w22 0)
             (<= w23 1) (>= w23 0)
             (<= w30 1) (>= w30 0)
             (<= w31 1) (>= w31 0)
             (<= w32 1) (>= w32 0)
             (<= w33 1) (>= w33 0)
        )
)


; PIT MAPPING

;sum of the neighbour following the schema (North + East + South + West) 
;starting from bottom left (0, 0)


;if we knew the total number of pits in the environment we could add this constraint too
;(assert (= 4 (+ p00 (+ p01 (+ p02 (+ p03 (+ p10 (+ p11 (+ p12 (+ p13 (+ p20 (+ p21 (+ p22 (+ p23 (+ p30 (+ p31 (+ p32 p33) ))))))))))))))))


(assert (= 0 (+ p01 p10)))
(assert (and (<= 1 (+ p11 (+ p20 p00))) (>= 3 (+ p11 (+ p20 p00)))))
(assert (= 0 (+ p10 (+ p21 p30))))
(assert (= 0 (+ p20 p31)))
(assert (and (<= 1 (+ p02 (+ p11 p00))) (>= 3 (+ p02 (+ p11 p00)))))
(assert (= 0 (+ p12 (+ p21 (+ p10 p01)))))
(assert (and (<= 1 (+ p22 (+ p31 (+ p20 p11)))) (>= 4 (+ p22 (+ p31 (+ p20 p11))))))
(assert (and (<= 1 (+ p32 (+ p30 p21))) (>= 3 (+ p32 (+ p30 p21)))))
(assert (= 0 (+ p03 (+ p12 p01))))
(assert (and (<= 1 (+ p13 (+ p22 (+ p11 p02)))) (>= 4 (+ p13 (+ p22 (+ p11 p02))))))
(assert (and (<= 1 (+ p23 (+ p32 (+ p21 p12)))) (>= 4 (+ p23 (+ p32 (+ p21 p12))))))
(assert (and (<= 1 (+ p33 (+ p31 p22))) (>= 3 (+ p33 (+ p31 p22)))))
(assert (= 0 (+ p13 p02)))
(assert (and (<= 1 (+ p23 (+ p12 p03))) (>= 3 (+ p23 (+ p12 p03)))))
(assert (and (<= 1 (+ p33 (+ p22 p13))) (>= 3 (+ p33 (+ p22 p13)))))
(assert (and (<= 1 (+ p32 p23)) (>= 2 (+ p32 p23))))


; WUMPUS MAPPING


(assert (= 1 (+ w00 (+ w01 (+ w02 (+ w03 (+ w10 (+ w11 (+ w12 (+ w13 (+ w20 (+ w21 (+ w22 (+ w23 (+ w30 (+ w31 (+ w32 w33)))))))))))))))))


(assert (= 0 (+ w01 w10)))
(assert (= 0 (+ w11 (+ w20 w00))))
(assert (and (<= 1 (+ w10 (+ w21 w30))) (>= 3 (+ w10 (+ w21 w30)))))
(assert (= 0 (+ w20 w31)))
(assert (= 0 (+ w02 (+ w11 w00))))
(assert (= 0 (+ w12 (+ w21 (+ w10 w01)))))
(assert (= 0 (+ w22 (+ w31 (+ w20 w11)))))
(assert (and (<= 1 (+ w32 (+ w30 w21))) (>= 3 (+ w32 (+ w30 w21)))))
(assert (= 0 (+ w03 (+ w12 w01))))
(assert (= 0 (+ w13 (+ w22 (+ w11 w02)))))
(assert (= 0 (+ w23 (+ w32 (+ w21 w12)))))
(assert (= 0 (+ w33 (+ w31 w22))))
(assert (= 0 (+ w13 w02)))
(assert (= 0 (+ w23 (+ w12 w03))))
(assert (= 0 (+ w33 (+ w22 w13))))
(assert (= 0 (+ w32 w23)))



(check-sat)
(get-value (p00 p01 p02 p03 p10 p11 p12 p13 p20 p21 p22 p23 p30 p31 p32 p33 
            w00 w01 w02 w03 w10 w11 w12 w13 w20 w21 w22 w23 w30 w31 w32 w33))
(exit)

sat
((p00 0)
 (p01 0)
 (p02 0)
 (p03 0)
 (p10 0)
 (p11 1)
 (p12 0)
 (p13 0)
 (p20 0)
 (p21 0)
 (p22 1)
 (p23 1)
 (p30 0)
 (p31 0)
 (p32 1)
 (p33 0)
 (w00 0)
 (w01 0)
 (w02 0)
 (w03 0)
 (w10 0)
 (w11 0)
 (w12 0)
 (w13 0)
 (w20 0)
 (w21 0)
 (w22 0)
 (w23 0)
 (w30 1)
 (w31 0)
 (w32 0)
 (w33 0))


2. **Example 2**
    ```
    +---+---+---+---+
    |   |  B|S  |  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|S B|  B|S  |
    |   |   |   |   |
    +---+---+---+---+
    |  B|  B|S B|  B|
    |   |   |   |   |
    +---+---+---+---+
    |  B|  B|  B|   |
    | A |   |   |   |
    +---+---+---+---+
    ```

In [32]:
%%script z3 -in -smt2

(set-option :produce-models true)

(declare-const p00 Int)
(declare-const p01 Int)
(declare-const p02 Int)
(declare-const p03 Int)
(declare-const p10 Int)
(declare-const p11 Int)
(declare-const p12 Int)
(declare-const p13 Int)
(declare-const p20 Int)
(declare-const p21 Int)
(declare-const p22 Int)
(declare-const p23 Int)
(declare-const p30 Int)
(declare-const p31 Int)
(declare-const p32 Int)
(declare-const p33 Int)

(declare-const w00 Int)
(declare-const w01 Int)
(declare-const w02 Int)
(declare-const w03 Int)
(declare-const w10 Int)
(declare-const w11 Int)
(declare-const w12 Int)
(declare-const w13 Int)
(declare-const w20 Int)
(declare-const w21 Int)
(declare-const w22 Int)
(declare-const w23 Int)
(declare-const w30 Int)
(declare-const w31 Int)
(declare-const w32 Int)
(declare-const w33 Int)

(assert (and (<= p00 1) (>= p00 0)
             (<= p01 1) (>= p01 0)
             (<= p02 1) (>= p02 0)
             (<= p03 1) (>= p03 0)
             (<= p10 1) (>= p10 0)
             (<= p11 1) (>= p11 0)
             (<= p12 1) (>= p12 0)
             (<= p13 1) (>= p13 0)
             (<= p20 1) (>= p20 0)
             (<= p21 1) (>= p21 0)
             (<= p22 1) (>= p22 0)
             (<= p23 1) (>= p23 0)
             (<= p30 1) (>= p30 0)
             (<= p31 1) (>= p31 0)
             (<= p32 1) (>= p32 0)
             (<= p33 1) (>= p33 0)
        )
)

(assert (and (<= w00 1) (>= w00 0)
             (<= w01 1) (>= w01 0)
             (<= w02 1) (>= w02 0)
             (<= w03 1) (>= w03 0)
             (<= w10 1) (>= w10 0)
             (<= w11 1) (>= w11 0)
             (<= w12 1) (>= w12 0)
             (<= w13 1) (>= w13 0)
             (<= w20 1) (>= w20 0)
             (<= w21 1) (>= w21 0)
             (<= w22 1) (>= w22 0)
             (<= w23 1) (>= w23 0)
             (<= w30 1) (>= w30 0)
             (<= w31 1) (>= w31 0)
             (<= w32 1) (>= w32 0)
             (<= w33 1) (>= w33 0)
        )
)


; PIT MAPPING

;sum of the neighbour following the schema (North + East + South + West) 
;starting from bottom left (0, 0)


;if we knew the total number of pits in the environment we could add this constraint too
;(assert (= 4 (+ p00 (+ p01 (+ p02 (+ p03 (+ p10 (+ p11 (+ p12 (+ p13 (+ p20 (+ p21 (+ p22 (+ p23 (+ p30 (+ p31 (+ p32 p33) ))))))))))))))))


(assert (and (<= 1 (+ p01 p10)) (>= 2 (+ p01 p10))))
(assert (and (<= 1 (+ p11 (+ p20 p00))) (>= 3 (+ p11 (+ p20 p00)))))
(assert (and (<= 1 (+ p10 (+ p21 p30))) (>= 3 (+ p10 (+ p21 p30)))))
(assert (= 0 (+ p20 p31)))
(assert (and (<= 1 (+ p02 (+ p11 p00))) (>= 3 (+ p02 (+ p11 p00)))))
(assert (and (<= 1 (+ p12 (+ p21 (+ p10 p01)))) (>= 4 (+ p12 (+ p21 (+ p10 p01))))))
(assert (and (<= 1 (+ p22 (+ p31 (+ p20 p11)))) (>= 4 (+ p22 (+ p31 (+ p20 p11))))))
(assert (and (<= 1 (+ p32 (+ p30 p21))) (>= 3 (+ p32 (+ p30 p21)))))
(assert (and (<= 1 (+ p03 (+ p12 p01))) (>= 3 (+ p03 (+ p12 p01)))))
(assert (and (<= 1 (+ p13 (+ p22 (+ p11 p02)))) (>= 4 (+ p13 (+ p22 (+ p11 p02))))))
(assert (and (<= 1 (+ p23 (+ p32 (+ p21 p12)))) (>= 4 (+ p23 (+ p32 (+ p21 p12))))))
(assert (= 0 (+ p33 (+ p31 p22))))
(assert (= 0 (+ p13 p02)))
(assert (and (<= 1 (+ p23 (+ p12 p03))) (>= 3 (+ p23 (+ p12 p03)))))
(assert (= 0 (+ p33 (+ p22 p13))))
(assert (and (<= 1 (+ p32 p23)) (>= 2 (+ p32 p23))))


; WUMPUS MAPPING


(assert (= 1 (+ w00 (+ w01 (+ w02 (+ w03 (+ w10 (+ w11 (+ w12 (+ w13 (+ w20 (+ w21 (+ w22 (+ w23 (+ w30 (+ w31 (+ w32 w33)))))))))))))))))


(assert (= 0 (+ w01 w10)))
(assert (= 0 (+ w11 (+ w20 w00))))
(assert (= 0 (+ w10 (+ w21 w30))))
(assert (= 0 (+ w20 w31)))
(assert (= 0 (+ w02 (+ w11 w00))))
(assert (= 0 (+ w12 (+ w21 (+ w10 w01)))))
(assert (and (<= 1 (+ w22 (+ w31 (+ w20 w11)))) (>= 4 (+ w22 (+ w31 (+ w20 w11))))))
(assert (= 0 (+ w32 (+ w30 w21))))
(assert (= 0 (+ w03 (+ w12 w01))))
(assert (and (<= 1 (+ w13 (+ w22 (+ w11 w02)))) (>= 4 (+ w13 (+ w22 (+ w11 w02))))))
(assert (= 0 (+ w23 (+ w32 (+ w21 w12)))))
(assert (and (<= 1 (+ w33 (+ w31 w22))) (>= 3 (+ w33 (+ w31 w22)))))
(assert (= 0 (+ w13 w02)))
(assert (= 0 (+ w23 (+ w12 w03))))
(assert (and (<= 1 (+ w33 (+ w22 w13))) (>= 3 (+ w33 (+ w22 w13)))))
(assert (= 0 (+ w32 w23)))

(check-sat)
(get-value (p00 p01 p02 p03 p10 p11 p12 p13 p20 p21 p22 p23 p30 p31 p32 p33 
            w00 w01 w02 w03 w10 w11 w12 w13 w20 w21 w22 w23 w30 w31 w32 w33))
(exit)

sat
((p00 0)
 (p01 1)
 (p02 0)
 (p03 0)
 (p10 0)
 (p11 1)
 (p12 1)
 (p13 0)
 (p20 0)
 (p21 0)
 (p22 0)
 (p23 0)
 (p30 1)
 (p31 0)
 (p32 1)
 (p33 0)
 (w00 0)
 (w01 0)
 (w02 0)
 (w03 0)
 (w10 0)
 (w11 0)
 (w12 0)
 (w13 0)
 (w20 0)
 (w21 0)
 (w22 1)
 (w23 0)
 (w30 0)
 (w31 0)
 (w32 0)
 (w33 0))


Now you should consider the case that only partial information is available, i.e. for some cells you don't have any information (below they're marked with `?`). This means that you can have several possible configurations (i.e. *models* in logic terms) corresponding to the available informations.

You should encode the configuration below (it's the first example with some unknown cells) and verify whether the first model the solver returns is the same as the one found for the first example.

```
+---+---+---+---+
|   |  B|  ?|  ?|
|   |   |   |   |
+---+---+---+---+
|   |  B|  ?|  B|
|   |   |   |   |
+---+---+---+---+
|  B|  ?|  ?|  ?|
|   |   |   |   |
+---+---+---+---+
|   |  B|S  |   |
| A |   |   |   |
+---+---+---+---+
```


In [33]:
%%script z3 -in -smt2

(set-option :produce-models true)

(declare-const p00 Int)
(declare-const p01 Int)
(declare-const p02 Int)
(declare-const p03 Int)
(declare-const p10 Int)
(declare-const p11 Int)
(declare-const p12 Int)
(declare-const p13 Int)
(declare-const p20 Int)
(declare-const p21 Int)
(declare-const p22 Int)
(declare-const p23 Int)
(declare-const p30 Int)
(declare-const p31 Int)
(declare-const p32 Int)
(declare-const p33 Int)

(declare-const w00 Int)
(declare-const w01 Int)
(declare-const w02 Int)
(declare-const w03 Int)
(declare-const w10 Int)
(declare-const w11 Int)
(declare-const w12 Int)
(declare-const w13 Int)
(declare-const w20 Int)
(declare-const w21 Int)
(declare-const w22 Int)
(declare-const w23 Int)
(declare-const w30 Int)
(declare-const w31 Int)
(declare-const w32 Int)
(declare-const w33 Int)

(assert (and (<= p00 1) (>= p00 0)
             (<= p01 1) (>= p01 0)
             (<= p02 1) (>= p02 0)
             (<= p03 1) (>= p03 0)
             (<= p10 1) (>= p10 0)
             (<= p11 1) (>= p11 0)
             (<= p12 1) (>= p12 0)
             (<= p13 1) (>= p13 0)
             (<= p20 1) (>= p20 0)
             (<= p21 1) (>= p21 0)
             (<= p22 1) (>= p22 0)
             (<= p23 1) (>= p23 0)
             (<= p30 1) (>= p30 0)
             (<= p31 1) (>= p31 0)
             (<= p32 1) (>= p32 0)
             (<= p33 1) (>= p33 0)
        )
)

(assert (and (<= w00 1) (>= w00 0)
             (<= w01 1) (>= w01 0)
             (<= w02 1) (>= w02 0)
             (<= w03 1) (>= w03 0)
             (<= w10 1) (>= w10 0)
             (<= w11 1) (>= w11 0)
             (<= w12 1) (>= w12 0)
             (<= w13 1) (>= w13 0)
             (<= w20 1) (>= w20 0)
             (<= w21 1) (>= w21 0)
             (<= w22 1) (>= w22 0)
             (<= w23 1) (>= w23 0)
             (<= w30 1) (>= w30 0)
             (<= w31 1) (>= w31 0)
             (<= w32 1) (>= w32 0)
             (<= w33 1) (>= w33 0)
        )
)

; PIT MAPPING

;sum of the neighbour following the schema (North + East + South + West) 
;starting from bottom left (0, 0)


;if we knew the total number of pits in the environment we could add this constraint too
;(assert (= 4 (+ p00 (+ p01 (+ p02 (+ p03 (+ p10 (+ p11 (+ p12 (+ p13 (+ p20 (+ p21 (+ p22 (+ p23 (+ p30 (+ p31 (+ p32 p33) ))))))))))))))))


(assert (= 0 (+ p01 p10)))
(assert (and (<= 1 (+ p11 (+ p20 p00))) (>= 3 (+ p11 (+ p20 p00)))))
(assert (= 0 (+ p10 (+ p21 p30))))
(assert (= 0 (+ p20 p31)))
(assert (and (<= 1 (+ p02 (+ p11 p00))) (>= 3 (+ p02 (+ p11 p00)))))
;(assert (= 0 (+ p12 (+ p21 (+ p10 p01)))))
;(assert (and (<= 1 (+ p22 (+ p31 (+ p20 p11)))) (>= 4 (+ p22 (+ p31 (+ p20 p11))))))
;(assert (and (<= 1 (+ p32 (+ p30 p21))) (>= 3 (+ p32 (+ p30 p21)))))
(assert (= 0 (+ p03 (+ p12 p01))))
(assert (and (<= 1 (+ p13 (+ p22 (+ p11 p02)))) (>= 4 (+ p13 (+ p22 (+ p11 p02))))))
;(assert (and (<= 1 (+ p23 (+ p32 (+ p21 p12)))) (>= 4 (+ p23 (+ p32 (+ p21 p12))))))
(assert (and (<= 1 (+ p33 (+ p31 p22))) (>= 3 (+ p33 (+ p31 p22)))))
(assert (= 0 (+ p13 p02)))
(assert (and (<= 1 (+ p23 (+ p12 p03))) (>= 3 (+ p23 (+ p12 p03)))))
;(assert (and (<= 1 (+ p33 (+ p22 p13))) (>= 3 (+ p33 (+ p22 p13)))))
;(assert (and (<= 1 (+ p32 p23)) (>= 2 (+ p32 p23))))


; WUMPUS MAPPING


(assert (= 1 (+ w00 (+ w01 (+ w02 (+ w03 (+ w10 (+ w11 (+ w12 (+ w13 (+ w20 (+ w21 (+ w22 (+ w23 (+ w30 (+ w31 (+ w32 w33)))))))))))))))))


(assert (= 0 (+ w01 w10)))
(assert (= 0 (+ w11 (+ w20 w00))))
(assert (and (<= 1 (+ w10 (+ w21 w30))) (>= 3 (+ w10 (+ w21 w30)))))
(assert (= 0 (+ w20 w31)))
(assert (= 0 (+ w02 (+ w11 w00))))
;(assert (= 0 (+ w12 (+ w21 (+ w10 w01)))))
;(assert (= 0 (+ w22 (+ w31 (+ w20 w11)))))
;(assert (and (<= 1 (+ w32 (+ w30 w21))) (>= 3 (+ w32 (+ w30 w21)))))
(assert (= 0 (+ w03 (+ w12 w01))))
(assert (= 0 (+ w13 (+ w22 (+ w11 w02)))))
;(assert (= 0 (+ w23 (+ w32 (+ w21 w12)))))
(assert (= 0 (+ w33 (+ w31 w22))))
(assert (= 0 (+ w13 w02)))
(assert (= 0 (+ w23 (+ w12 w03))))
;(assert (= 0 (+ w33 (+ w22 w13))))
;(assert (= 0 (+ w32 w23)))


(check-sat)
(get-value (p00 p01 p02 p03 p10 p11 p12 p13 p20 p21 p22 p23 p30 p31 p32 p33 
            w00 w01 w02 w03 w10 w11 w12 w13 w20 w21 w22 w23 w30 w31 w32 w33))
(exit)

sat
((p00 0)
 (p01 0)
 (p02 0)
 (p03 0)
 (p10 0)
 (p11 1)
 (p12 0)
 (p13 0)
 (p20 0)
 (p21 0)
 (p22 1)
 (p23 1)
 (p30 0)
 (p31 0)
 (p32 0)
 (p33 0)
 (w00 0)
 (w01 0)
 (w02 0)
 (w03 0)
 (w10 0)
 (w11 0)
 (w12 0)
 (w13 0)
 (w20 0)
 (w21 1)
 (w22 0)
 (w23 0)
 (w30 0)
 (w31 0)
 (w32 0)
 (w33 0))


Example of a unsatisfiable world. A breeze in (1, 1) indicates a pit, but since the cell in (3, 2) has no breeze ther can not be a pit in (3, 1).

```
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |  B|  ?|
|   |   |   |   |
+---+---+---+---+
|   |   |   |  ?|
| A |   |   |   |
+---+---+---+---+
```



In [34]:
%%script z3 -in -smt2

(set-option :produce-models true)

(declare-const p00 Int)
(declare-const p01 Int)
(declare-const p02 Int)
(declare-const p03 Int)
(declare-const p10 Int)
(declare-const p11 Int)
(declare-const p12 Int)
(declare-const p13 Int)
(declare-const p20 Int)
(declare-const p21 Int)
(declare-const p22 Int)
(declare-const p23 Int)
(declare-const p30 Int)
(declare-const p31 Int)
(declare-const p32 Int)
(declare-const p33 Int)

(assert (and (<= p00 1) (>= p00 0)
             (<= p01 1) (>= p01 0)
             (<= p02 1) (>= p02 0)
             (<= p03 1) (>= p03 0)
             (<= p10 1) (>= p10 0)
             (<= p11 1) (>= p11 0)
             (<= p12 1) (>= p12 0)
             (<= p13 1) (>= p13 0)
             (<= p20 1) (>= p20 0)
             (<= p21 1) (>= p21 0)
             (<= p22 1) (>= p22 0)
             (<= p23 1) (>= p23 0)
             (<= p30 1) (>= p30 0)
             (<= p31 1) (>= p31 0)
             (<= p32 1) (>= p32 0)
             (<= p33 1) (>= p33 0)
        )
)

; PIT MAPPING

;sum of the neighbour following the schema (North + East + South + West) 
;starting from bottom left (0, 0)


;if we knew the total number of pits in the environment we could add this constraint too
;(assert (= 4 (+ p00 (+ p01 (+ p02 (+ p03 (+ p10 (+ p11 (+ p12 (+ p13 (+ p20 (+ p21 (+ p22 (+ p23 (+ p30 (+ p31 (+ p32 p33) ))))))))))))))))

;p00
;(assert (and (<= 1 (+ p01 p10)) (>= 2 (+ p01 p10))))
(assert (= 0 (+ p01 p10)))
;p01
;(assert (and (<= 1 (+ p11 (+ p20 p00))) (>= 3 (+ p11 (+ p20 p00)))))
(assert (= 0 (+ p11 (+ p20 p00))))
;p02
;(assert (and (<= 1 (+ p10 (+ p21 p30))) (>= 3 (+ p10 (+ p21 p30)))))
(assert (= 0 (+ p10 (+ p21 p30))))
;p03
;(assert (and (<= 1 (+ p20 p31)) (>= 2 (+ p20 p31))))
(assert (= 0 (+ p20 p31)))
;p10
;(assert (and (<= 1 (+ p02 (+ p11 p00))) (>= 3 (+ p02 (+ p11 p00)))))
(assert (= 0 (+ p02 (+ p11 p00))))
;p11
;(assert (and (<= 1 (+ p12 (+ p21 (+ p10 p01)))) (>= 4 (+ p12 (+ p21 (+ p10 p01))))))
(assert (= 0 (+ p12 (+ p21 (+ p10 p01)))))
;p12
(assert (and (<= 1 (+ p22 (+ p31 (+ p20 p11)))) (>= 4 (+ p22 (+ p31 (+ p20 p11))))))
;(assert (= 0 (+ p22 (+ p31 (+ p20 p11)))))
;p13
;(assert (and (<= 1 (+ p32 (+ p30 p21))) (>= 3 (+ p32 (+ p30 p21)))))
;(assert (= 0 (+ p32 (+ p30 p21))))
;p20
;(assert (and (<= 1 (+ p03 (+ p12 p01))) (>= 3 (+ p03 (+ p12 p01)))))
(assert (= 0 (+ p03 (+ p12 p01))))
;p21
;(assert (and (<= 1 (+ p13 (+ p22 (+ p11 p02)))) (>= 4 (+ p13 (+ p22 (+ p11 p02))))))
(assert (= 0 (+ p13 (+ p22 (+ p11 p02)))))
;p22
;(assert (and (<= 1 (+ p23 (+ p32 (+ p21 p12)))) (>= 4 (+ p23 (+ p32 (+ p21 p12))))))
(assert (= 0 (+ p23 (+ p32 (+ p21 p12)))))
;p23
;(assert (and (<= 1 (+ p33 (+ p31 p22))) (>= 3 (+ p33 (+ p31 p22)))))
;(assert (= 0(+ p33 (+ p31 p22))))
;p30
;(assert (and (<= 1 (+ p13 p02)) (>= 2 (+ p13 p02))))
(assert (= 0 (+ p13 p02)))
;p31
;(assert (and (<= 1 (+ p23 (+ p12 p03))) (>= 3 (+ p23 (+ p12 p03)))))
(assert (= 0 (+ p23 (+ p12 p03))))
;p32
;(assert (and (<= 1 (+ p33 (+ p22 p13))) (>= 3 (+ p33 (+ p22 p13)))))
(assert (= 0 (+ p33 (+ p22 p13))))
;p33
;(assert (and (<= 1 (+ p32 p23)) (>= 2 (+ p32 p23))))
(assert (= 0 (+ p32 p23)))

(check-sat)
;(get-value (p00 p01 p02 p03 p10 p11 p12 p13 p20 p21 p22 p23 p30 p31 p32 p33))
(exit)

unsat


Using just SMT-LIB is not easy to find all the models; so we'll use the Z3 Python API to do so.

## Using the Python API

Z3 provides APIs (bindings) for [different programming languages](https://github.com/Z3Prover/z3#z3-bindings). Python binding is a convenient interface to Z3 which is already bundled with the `smt` conda environment. A brief tutorial is available on [Z3 API in Python](http://ericpony.github.io/z3py-tutorial/guide-examples.htm) document (source available on [github](https://github.com/ericpony/z3py-tutorial)), a more detailed tutorial is available on [Programming Z3](http://theory.stanford.edu/~nikolaj/programmingz3.html), and the full API is described on [API documentation](http://z3prover.github.io/api/html/namespacez3py.html).

For example the SAT example from the [tutorial](http://theory.stanford.edu/~nikolaj/programmingz3.html#sec-logical-interface) can be run with:

In [35]:
from z3 import *
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(Or(Tie, Shirt), 
    Or(Not(Tie), Shirt), 
    Or(Not(Tie), Not(Shirt)))
print(s.check())
print(s.model())

sat
[Tie = False, Shirt = True]


Note that you don't need the `%%z3` magic since you're using Python code which is evaluated directly by the kernel.

### Examples using Python API

Convert the solutions to the previous exercises using Python API. For this exercise you can write your code in cells below or separate files.

### Enumerating all models

Using the API you can write a procedure to enumerate all the models. The key to avoid the repetition of models that have been already enumerated is to add a formulae that falsify the models already discovered in such a way that they would be avoided in the search.

Consider that a model is a set of variable assignments; therefore a formula that is blocking the model is the negation of the value assignments; e.g.:

In [36]:
from z3 import *
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(Xor(Tie, Shirt))

while s.check() == sat:
    print(s.model())
    s.add(Or([v() != s.model()[v] for v in s.model()]))

[Tie = True, Shirt = False]
[Tie = False, Shirt = True]


Note the `v()` reference to the variable from the model and be aware that the above technique doesn't work with functions and arrays, see [this](https://stackoverflow.com/a/11869410) stackoverflow answer for details.

### Adjancecy matrix blueprint

For 4x4 gridmap:

Cells with perception: 1st line uncommented and 2nd line commented

Cells with no perception: 1st line commented and 2nd line uncommented

Cells which are unknown: both lines commented

In [ ]:
;p00
(assert (and (<= 1 (+ p01 p10)) (>= 2 (+ p01 p10))))
;(assert (= 0 (+ p01 p10)))
;p01
(assert (and (<= 1 (+ p11 (+ p20 p00))) (>= 3 (+ p11 (+ p20 p00)))))
;(assert (= 0 (+ p11 (+ p20 p00))))
;p02
(assert (and (<= 1 (+ p10 (+ p21 p30))) (>= 3 (+ p10 (+ p21 p30)))))
;(assert (= 0 (+ p10 (+ p21 p30))))
;p03
(assert (and (<= 1 (+ p20 p31)) (>= 2 (+ p20 p31))))
;(assert (= 0 (+ p20 p31)))
;p10
(assert (and (<= 1 (+ p02 (+ p11 p00))) (>= 3 (+ p02 (+ p11 p00)))))
;(assert (= 0 (+ p02 (+ p11 p00))))
;p11
(assert (and (<= 1 (+ p12 (+ p21 (+ p10 p01)))) (>= 4 (+ p12 (+ p21 (+ p10 p01))))))
;(assert (= 0 (+ p12 (+ p21 (+ p10 p01)))))
;p12
(assert (and (<= 1 (+ p22 (+ p31 (+ p20 p11)))) (>= 4 (+ p22 (+ p31 (+ p20 p11))))))
;(assert (= 0 (+ p22 (+ p31 (+ p20 p11)))))
;p13
(assert (and (<= 1 (+ p32 (+ p30 p21))) (>= 3 (+ p32 (+ p30 p21)))))
;(assert (= 0 (+ p32 (+ p30 p21))))
;p20
(assert (and (<= 1 (+ p03 (+ p12 p01))) (>= 3 (+ p03 (+ p12 p01)))))
;(assert (= 0 (+ p03 (+ p12 p01))))
;p21
(assert (and (<= 1 (+ p13 (+ p22 (+ p11 p02)))) (>= 4 (+ p13 (+ p22 (+ p11 p02))))))
;(assert (= 0 (+ p13 (+ p22 (+ p11 p02)))))
;p22
(assert (and (<= 1 (+ p23 (+ p32 (+ p21 p12)))) (>= 4 (+ p23 (+ p32 (+ p21 p12))))))
;(assert (= 0 (+ p23 (+ p32 (+ p21 p12)))))
;p23
(assert (and (<= 1 (+ p33 (+ p31 p22))) (>= 3 (+ p33 (+ p31 p22)))))
;(assert (= 0(+ p33 (+ p31 p22))))
;p30
(assert (and (<= 1 (+ p13 p02)) (>= 2 (+ p13 p02))))
;(assert (= 0 (+ p13 p02)))
;p31
(assert (and (<= 1 (+ p23 (+ p12 p03))) (>= 3 (+ p23 (+ p12 p03)))))
;(assert (= 0 (+ p23 (+ p12 p03))))
;p32
(assert (and (<= 1 (+ p33 (+ p22 p13))) (>= 3 (+ p33 (+ p22 p13)))))
;(assert (= 0 (+ p33 (+ p22 p13))))
;p33
(assert (and (<= 1 (+ p32 p23)) (>= 2 (+ p32 p23))))
;(assert (= 0 (+ p32 p23)))